In [ ]:
def word_definition_quiz(word,definition_options):
    print('Welcome to the word quiz.')
    print(f'What is the correct definition of this word: {word}\n')
    
    for num,option in enumerate(definition_options):
        print('\n')
        print(f'Definition #{num} is: {option}')
    
    print('\n')
    num_choice = input("What is your choice? (Return the single number option)")
    return definition_options[int(num_choice)]

In [ ]:
# word_definition_quiz("hello",['greeting','farewell','sandwich'])

In [ ]:
function_json = {'type':'function',
                 'function':{
                     'name':'word_definition_quiz',
                     'parameters':{
                         'type':'object',
                          'properties':{
                          'word': {'type':'string','description':'A random word'},
                          'definition_options': {'type':'array',
                                                 'items':{'type':'string'},
                                                 'description':"A Python list of strings of definitions, where only one is the correct definition for the word."
                                                }
                          },
                         'required': ['word','defintion_options']
                     }
                 }
                }
                    
                     

In [ ]:
from openai import OpenAI

In [ ]:
client = OpenAI()

In [ ]:
instructions = "You help create a quiz for checking definitions of words, providing a word and then multiple definition options, where only one option is correct."

In [ ]:
assistant = client.beta.assistants.create(instructions=instructions,
  model="gpt-3.5-turbo",
  tools=[function_json]
)

In [ ]:
thread = client.beta.threads.create()

In [ ]:
message = client.beta.threads.messages.create(thread_id=thread.id,
                                              role='user',
                                              content='Create a new quiz question word and definition list. Then let me know if the student answer recieved is correct'
                                             )
                                              

In [ ]:
import time

def wait_on_run(run, thread):
    while run.status == "queued" or run.status == "in_progress":
        run = client.beta.threads.runs.retrieve(thread_id=thread.id,run_id=run.id,)
        time.sleep(0.5)
        
    return run

In [ ]:
run = client.beta.threads.runs.create(thread_id=thread.id,assistant_id=assistant.id)

In [ ]:
wait_on_run(run,thread)

In [ ]:
run.status

In [ ]:
run = client.beta.threads.runs.retrieve(thread_id=thread.id,run_id=run.id,)

In [ ]:
run.status

In [ ]:
tool_call = run.required_action.submit_tool_outputs.tool_calls[0]

In [ ]:
name = tool_call.function.name

In [ ]:
import json
arguments = json.loads(tool_call.function.arguments)

In [ ]:
arguments['word']

In [ ]:
arguments['definition_options']

In [ ]:
response = word_definition_quiz(arguments['word'],arguments['definition_options'])

In [ ]:
response

In [ ]:
tool_call

In [ ]:
run = client.beta.threads.runs.submit_tool_outputs(thread_id=thread.id,run_id=run.id,
    tool_outputs=[{"tool_call_id": tool_call.id,
                   "output": json.dumps(response)
                  }],
)

In [ ]:
run = wait_on_run(run, thread)

In [ ]:
run.status

In [ ]:
messages = client.beta.threads.messages.list(thread.id)

In [ ]:
for thread_message in messages:
    print(thread_message.content[0].text.value)
    print('\n')

In [ ]:
my_assistants = client.beta.assistants.list(
    order="desc",
    limit="20",
)
response = client.beta.assistants.delete(my_assistants.data[0].id)
print(response)